In [28]:
# %pip install --force-reinstall --no-deps torch==2.2.2 torchvision==0.17.2

In [29]:
# %pip install statsforecast mlforecast pandas polars pyarrow holidays lightgbm neuralforecast

In [30]:
import pandas as pd
from utils.constants import FREQ_DAILY, FREQ_MONTHLY, FAMILY_BASELINE, FAMILY_STATISTICAL, FAMILY_MACHINE_LEARNING, FAMILY_DEEP_LEARNING, SPLIT_VAL
from utils.preprocessing import load_daily_data, load_monthly_data
from utils.baseline import run_baseline_forecast_daily, run_baseline_forecast_monthly
from utils.statistical import run_statistical_forecast_daily, run_statistical_forecast_monthly
from utils.machine_learning import run_machine_learning_forecast_daily, run_machine_learning_forecast_monthly
from utils.deep_learning import run_deep_learning_forecast_daily, run_deep_learning_forecast_monthly
from utils.metrics import calculate_metrics

### Loading Data:

In [31]:
daily_train, daily_val, daily_test = load_daily_data(use_existing=True)
monthly_train, monthly_val, monthly_test = load_monthly_data(use_existing=True)

### Forecasts:

##### Baseline Forecast:

In [32]:
base_daily_val, base_daily_test = run_baseline_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
base_monthly_val, base_monthly_test = run_baseline_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Statistical Forecast:

In [33]:
statis_daily_val, statis_daily_test = run_statistical_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
statis_monthly_val, statis_monthly_test = run_statistical_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Machine Learning Forecast:

In [34]:
ml_daily_val, ml_daily_test = run_machine_learning_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
ml_monthly_val, ml_monthly_test = run_machine_learning_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

##### Deep Learning Forecast:

In [35]:
dl_daily_val, dl_daily_test = run_deep_learning_forecast_daily(daily_train, daily_val, daily_test, use_existing=True)
dl_monthly_val, dl_monthly_test = run_deep_learning_forecast_monthly(monthly_train, monthly_val, monthly_test, use_existing=True)

### Metrics:

##### Baseline Forecast:

In [36]:
base_metrics = pd.concat([
    calculate_metrics(base_daily_val, FAMILY_BASELINE, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(base_monthly_val, FAMILY_BASELINE, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
])

##### Statistical Forecast:

In [37]:
stat_metrics = pd.concat([
    calculate_metrics(statis_daily_val, FAMILY_STATISTICAL, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(statis_monthly_val, FAMILY_STATISTICAL, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
])

##### Machine Learning Forecast:

In [38]:
ml_metrics = pd.concat([
    calculate_metrics(ml_daily_val, FAMILY_MACHINE_LEARNING, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(ml_monthly_val, FAMILY_MACHINE_LEARNING, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
])

##### Deep Learning Forecast:

In [39]:
dl_metrics = pd.concat([
    calculate_metrics(dl_daily_val, FAMILY_DEEP_LEARNING, FREQ_DAILY, SPLIT_VAL, use_existing=True),
    calculate_metrics(dl_monthly_val, FAMILY_DEEP_LEARNING, FREQ_MONTHLY, SPLIT_VAL, use_existing=True)
])